In [1]:
import location_functions as L
import os
import re
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bs4 import BeautifulSoup
import geocoder
import requests
import folium
from folium.plugins import MousePosition
from folium.plugins import Draw
from geopy.distance import distance as getDistance
load_dotenv()

dbName = 'companies'

db, boston = L.connectCollection(dbName,'companies')
db, start = L.connectCollection(dbName,'startups')
db, old_comp = L.connectCollection(dbName,'old_comp')
db, starbucks = L.connectCollection(dbName,'starbucks')
db, schools = L.connectCollection(dbName,'schools')
db, night = L.connectCollection(dbName,'night')

1. De angel.co sacamos un csv filtrado de startups en Boston que han ganado entre 2 y 100 M   
2. Filtramos las startups que hayan ganado mas de 10 M
3. Obtenemos la direccion de todas las startups en Boston (nombre y estado)
4. Insertamos la posicion en la coleccion de startups de mongodb 

In [3]:
startups = list(start.find())
startups = [startups[s] for s in range(len(startups)-1) if int(startups[s]["Total Raised"])>1000000]

for startup in startups[:-1]:
    x = L.place_request(startup["Name"]+",MA")
    try:
        lat = x["raw"]["geometry"]["location"]["lat"]
        lng = x["raw"]["geometry"]["location"]["lng"]
        L.createLocation(lng,lat,start,startup)
    except:
        pass